<a href="https://colab.research.google.com/github/SergeyKamenshchikov/CORONA/blob/master/spacy_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [2]:
!pip3 install allennlp==1.0.0 allennlp-models==1.0.0

     |████████████████████████████████| 481kB 4.4MB/s 
     |████████████████████████████████| 286kB 13.5MB/s 
     |████████████████████████████████| 266kB 14.7MB/s 
     |████████████████████████████████| 753.2MB 19kB/s 
     |████████████████████████████████| 317kB 40.3MB/s 
     |████████████████████████████████| 675kB 39.7MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 890kB 36.3MB/s 
     |████████████████████████████████| 1.1MB 42.6MB/s 
     |████████████████████████████████| 3.8MB 34.6MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.16.0-cp36-cp36m-linux_x86_64.whl size=3321641 sha256=251f92e6ca75494f1a6e1e4d8830f134a30c05a042f0b0dec2285202b8921580
  Stored in directory: /root/.cache/pip/wheels/64/a9/43/bc5e0463deeec89dfca928a2a64595f1bdb520c891f6fbd09c
  Created wheel for overrides: filename=overrides-3.0.0-cp36-none-any.whl size=5669 sha256=d246177b59a5a5050c977db0d57b89c02455d1080018bf74c67b3e064c0b4349
  Stored in di

In [6]:
import nltk
nltk.download('stopwords')

!wget https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
--2020-09-08 12:10:26--  https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1345947288 (1.3G) [application/x-tar]
Saving to: ‘coref-spanbert-large-2020.02.27.tar.gz’

coref-spanbert-larg 100%[===================>]   1.25G  14.5MB/s    in 85s     

2020-09-08 12:11:52 (15.1 MB/s) - ‘coref-spanbert-large-2020.02.27.tar.gz’ saved [1345947288/1345947288]



In [37]:
# import libraries
import string
from textblob import TextBlob
from gensim.summarization.summarizer import summarize

from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')

from bs4 import BeautifulSoup
import numpy as np
import requests

from allennlp.predictors.predictor import Predictor
import allennlp_models.coref

#!wget https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz
predictor = Predictor.from_path('./coref-spanbert-large-2020.02.27.tar.gz')

In [33]:
# customer defined functions
def get_unique_text(document):
    unique_sentences = []
    for sentence in [sent.raw for sent in TextBlob(document).sentences]:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return ' '.join(unique_sentences)

def get_summary(rawtext,readtime):
    sentences = int(readtime/(np.median([len(i.split()) for i in nltk.sent_tokenize(rawtext)])/200))
    ratio = sentences/len(nltk.sent_tokenize(rawtext))
    txt = summarize(rawtext, ratio=ratio)

    z = 0
    output = []
    for i in nltk.sent_tokenize(txt):
        z = z+1
        output.append('\n\n<hr>' + str(z) + '. ' + str(i))

    txt = ' '.join(output)
    return txt    

In [29]:
htmlString = requests.get('https://medium.com/@sergeykamenshchikov/nlp-for-composition-of-value-tree-d8495534ebd5')

soup = BeautifulSoup(htmlString.content, 'html.parser')
paragraphs = soup.find_all(['p', 'article', 'section', 'title', 'h1', 'h2', 'h3'])
rawtext = ' '.join([i.text for i in paragraphs])
rawtext=get_unique_text(rawtext)

print(rawtext)

NLP for composition of Value Tree | by Sergey Kamenshchikov | Medium Responses  Sergey KamenshchikovFollowSep 29, 2019 · 6 min readTree of Keywords: graph analysis of semantics attractorsApplication domain. According to the WHO depressive disorder is the leading cause of disability worldwide. It is found that generating mechanism lies in rumination: long-term stressful attachment to the problem and underlying Super Value in danger. Efforts of predictive pattern recognition are made in several fields - Machine Learning is among them.Previous research. According to Al-Mosaiwi, single words-absolutisms may be used as predictors of cognitive errors following rumination. However it is recognized that analysis of single word frequencies is not enough. The reason is simple: random mixing of word collection does not affect the relative frequency, but removes information — bag of words effect. This means that the semantic source of rumination, value, is outside the scope. Besides, according to 

In [41]:
#coreference resolution
%%time
span_dict=predictor.predict(document=rawtext)
words=span_dict['document']

original_document=span_dict['document']
clusters=span_dict['clusters']

for cluster in clusters:
    first_element=cluster[0]
    head=original_document[first_element[0]:first_element[1]+1] #get head of cluster
    if len(head)==1 and head[0].lower() in en_stopwords: #check if pronoun
        pass
    else:
        for element in cluster:
            for index in range(element[0],element[1]+1):
                words[index]='' #make all elements empty to save indexes of document words
        for element in cluster:
            words[element[0]]="".join([" "+w if not w.startswith("'") and w not in string.punctuation else w for w in head]).strip()

document="".join([" "+w if not w.startswith("'") and w!='' and w not in string.punctuation else w for w in words]).strip()            

CPU times: user 1min 15s, sys: 4.37 s, total: 1min 19s
Wall time: 1min 19s


In [40]:
final_summary = get_summary(document,2)
print(final_summary)



<hr>1. NLP for composition of Value Tree| by Sergey Kamenshchikov| Medium Responses   Sergey Kamenshchikov 29, 2019 · 6 min readTree of Keywords: graph analysis of semantics attractorsApplication domain. 

<hr>2. According to Al- Mosaiwi, single words- absolutisms may be used as predictors of cognitive errors following rumination. 

<hr>3. The Researchers from MIT have developed and trained neural network, which predicts depression rate in clinical interviews with precision of 71% and recall of 83%. 

<hr>4. However The Researchers from MIT admit that neural network, which predicts depression rate in clinical interviews with precision of 71% and recall of 83% discovers cognitive impairments as well, ignoring cognition structure. 

<hr>5. Pathological states like depression correspond to destructive associative schemes: specific graph clusters which trigger ruminative loops. 

<hr>6. We assume that specific graph clusters which trigger ruminative loops may have high betweennes central